In [79]:
import pandas as pd
import time
from bs4 import BeautifulSoup
import requests
import warnings
from fake_useragent import UserAgent
import warnings

class Webscraper():
    
    def __init__(self, urls):
        self.data = None
        self.urls = list(urls)
        self.smm_dict = None
         
    def scrap_smm_info_from_pages(self):

        social_media_patterns = ['www.facebook.com', 'www.instagram.com', 'www.linkedin.com', 'www.twitter.com',
        'facebook.com', 'instagram.com', 'linkedin.com', 'twitter.com', 'x.com']
        ua = UserAgent()
        fake_headers = ua.chrome
        smm_dict = {}
        warnings.filterwarnings("ignore")
        websites_to_scrap = [x.lower() for x in self.urls]
        for smm_find in websites_to_scrap:
            smm_links = set()
            try:
                response2 = requests.get(smm_find, timeout=8, headers={'User-Agent' : fake_headers}, verify=False)
                soupy = BeautifulSoup(response2.text, 'html.parser')
                body = soupy.find('body')

                if body is not None:
                    href_values = [link.get('href') for link in body.find_all('a') if link.get('href')]

                    for href in href_values:
                        for pattern in social_media_patterns:
                            if pattern in href:
                                smm_links.add(href)

            except Exception as e:
                print(f'Hi, {smm_find} seems to be invalid, we are going to the next one!')
                continue

            if smm_links is not None and len(smm_links) > 0:
                smm_dict[smm_find] = list(smm_links)
            else:
                print(f'No Social Media Accounts were found for {smm_find}.')


        self.smm_dict = smm_dict
        return smm_dict

    
    def create_df_cleaned_from_smm(self, smm_dict):
        df_of_smm_opt = pd.DataFrame(smm_dict.items(), columns=['Website', 'Social Media Links'])

        platforms = ['Facebook', 'Instagram', 'LinkedIn', 'Twitter']
        print(df_of_smm_opt)
        for platform in platforms:
            df_of_smm_opt[platform] = df_of_smm_opt['Social Media Links'].apply(
                lambda links: next((link for link in links if platform.lower() in link.lower()), None)
            )

        df_of_smm_opt = df_of_smm_opt.drop('Social Media Links', axis=1)
      
        return df_of_smm_opt

In [80]:
urls = ['https://faMilyfriendsvet.com', 'https://bluepearlvet.com', 'https://www.westmichiganaeh.com', 'https://willowrunvetservices.com']

scrapper_object = Webscraper(urls)

scrapping_websites_method = scrapper_object.scrap_smm_info_from_pages()
data_frame = scrapper_object.create_df_cleaned_from_smm(scrapping_websites_method)
data_frame


                            Website   
0      https://familyfriendsvet.com  \
1          https://bluepearlvet.com   
2   https://www.westmichiganaeh.com   
3  https://willowrunvetservices.com   

                                  Social Media Links  
0  [https://www.facebook.com/familyfriendsvet/, h...  
1  [https://www.facebook.com/BluePearlPetHospital...  
2                   [https://www.facebook.com/AEHGR]  
3  [https://www.facebook.com/Willow-Run-Veterinar...  


,Website,Facebook,Instagram,LinkedIn,Twitter
0,https://familyfriendsvet.com,https://www.facebook.com/familyfriendsvet/,https://www.instagram.com/familyfriendsveterin...,None,None
1,https://bluepearlvet.com,https://www.facebook.com/BluePearlPetHospital/,https://www.instagram.com/bluepearlvet/,https://www.linkedin.com/company/bluepearl-vet...,https://twitter.com/bluepearlvet
2,https://www.westmichiganaeh.com,https://www.facebook.com/AEHGR,None,None,None
3,https://willowrunvetservices.com,https://www.facebook.com/Willow-Run-Veterinary...,None,None,None
